In [1]:
%use fuel(2.3.1)

In [2]:
import java.util.Stack

In [3]:
val envs = java.io.File("../../.env")
    .readLines()
    .map {
        it.split("=")[0] to it.split("=")[1].trim('"')
    }.toMap()

In [4]:
val session = envs.get("AOC_SESSION")
val year = 2021
val day = 18

In [5]:
fun getInput(year: Int, day: Int, session: String): String {
    val (_, _, result) = "https://adventofcode.com/$year/day/$day/input"
    .httpGet()
    .header("cookie" to "session=$session")
    .responseString()
        
    return result.get().trim()
}

In [6]:
fun submitAnswer(year: Int, day: Int, session: String, level: Int, answer: String): String {
    val (_, _, result) = Fuel
    .post(
        "https://adventofcode.com/$year/day/$day/answer", 
        parameters = listOf("level" to level, "answer" to answer))
    .header("cookie" to "session=$session")
    .responseString()
        
    return result.get()
}

In [7]:
val sample = """[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]"""

In [8]:
val input = getInput(year, day, session)

In [9]:
class Node{
    var num: Int = 0
    var left: Node? = null
    var right: Node? = null
    var parent: Node? = null
    
    constructor (num: Int) {
        this.num = num
    }
    
    constructor (str: String) {
        if (!str.startsWith("[")) {
            this.num = str.toInt()
        } else {
            val stk = Stack<Node>()
            var numberStart = -1
            for ((i, ch) in str.withIndex()) {      
                when (ch) {
                    '[' -> {
                        if (stk.isEmpty())
                            stk.add(this)
                        else
                            stk.add(Node(0))
                    }
                    ']' -> {
                        val node = if (numberStart != -1) {
                            Node(str.slice(numberStart until i))
                        } else {
                            stk.pop()!!
                        }
                        
                        node.parent = stk.peek()
                        if (node.parent!!.left == null)
                            node.parent!!.left = node
                        else 
                            node.parent!!.right = node
                        
                        numberStart = -1
                    }
                    ',' -> {
                        val node = if (numberStart != -1) {
                            Node(str.slice(numberStart until i))
                        } else {
                            stk.pop()!!
                        }
                        
                        node.parent = stk.peek()
                        node.parent!!.left = node
                        numberStart = -1
                    }
                    else -> {
                        if (numberStart == -1)
                            numberStart = i
                    }
                }
            }
        }
        
        this.reduce()
    }
    
    constructor (a: Node) {
        this.num = a.num
        if (a.left != null) {
            this.left = Node(a.left!!)
            this.left?.parent = this
        }
        if (a.right != null) {
            this.right = Node(a.right!!)
            this.right?.parent = this
        }
    }
    
    constructor (a: Node, b: Node) {
        this.left = Node(a)
        this.left!!.parent = this
        this.right = Node(b)
        this.right!!.parent = this
        this.reduce()
    }
    
    fun prevRegular(): Node? {
        if (this.parent == null)
            return null
        
        if (this == this.parent!!.left)
            return this.parent!!.prevRegular()
            
        var ans = this.parent!!.left
        while (ans!!.right != null)
            ans = ans.right
        
        return ans
    }
    
    fun nextRegular(): Node? {
        if (this.parent == null)
            return null
        
        if (this == this.parent!!.right)
            return this.parent!!.nextRegular()
            
        var ans = this.parent!!.right
        while (ans!!.left != null)
            ans = ans!!.left
        
        return ans
    }
    
    fun explode() {
        val lValue = this.left!!.num
        val rValue = this.right!!.num
        val prev = this.prevRegular()
        val next = this.nextRegular()
        
        if (prev != null)
            prev.num += lValue
        
        if (next != null)
            next.num += rValue
        
        this.left = null
        this.right = null
    }
    
    fun split() {
        val lValue = this.num / 2
        val rValue = this.num - lValue
        this.num = 0
        this.left = Node(lValue)
        this.left?.parent = this
        this.right = Node(rValue)
        this.right?.parent = this
    }
    
    fun checkExplosion(depth: Int, changed: BooleanArray) {
        if (changed[0])
            return
        
        if (depth >= 4 && this.left != null && this.left!!.left == null) {
            changed[0] = true
            this.explode()
            return
        }
        
        this.left?.checkExplosion(depth + 1, changed)
        this.right?.checkExplosion(depth + 1, changed)
    }
    
    fun checkSplit(changed: BooleanArray) {
        if (changed[0])
            return
        
        if (this.num >= 10) {
            changed[0] = true
            this.split()
            return
        }
        
        this.left?.checkSplit(changed)
        this.right?.checkSplit(changed)
    }
        
    fun reduce() {
        val changed = BooleanArray(1)
        this.checkExplosion(0, changed)
        if (!changed[0])
            this.checkSplit(changed)
            
        if (changed[0])
            this.reduce()
    }
    
    fun mag(): Int {
        if (this.left == null)
            return this.num 
        else 
            return 3 * this.left!!.mag() + 2 * this.right!!.mag()
    }
    
    operator fun plus(other: Node): Node {
        return Node(this, other)
    }
    
    override fun toString(): String {
        if (this.left == null) {
            return this.num.toString()
        } else {
            return "[" + this.left?.toString() + "," + this.right?.toString() + "]"
        }
    }
}

In [10]:
fun partOne(input: String): String {
    val nums = input.split("\n").map { Node(it) }
    var ans = nums.drop(1).fold(nums[0]) {acc, it -> acc + it}
    return ans.mag().toString()
}

In [11]:
partOne(sample)

4140

In [12]:
val partOneAns = partOne(input)
partOneAns

3756

In [ ]:
submitAnswer(year, day, session, 1, partOneAns)

In [13]:
fun partTwo(input: String): String {
    val nums = input.split("\n").map { Node(it) }
    var ans = 0
    
    for (i in 0 until nums.size)
        for (j in 0 until nums.size)
            if (i != j)
                ans = ans.coerceAtLeast((nums[i] + nums[j]).mag())
    
    return ans.toString()
}

In [14]:
partTwo(sample)

3993

In [15]:
val partTwoAns = partTwo(input)
partTwoAns

4585

In [ ]:
submitAnswer(year, day, session, 2, partTwoAns)